<a href="https://colab.research.google.com/github/gorovuha/text_generation/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim

In [ ]:
!pip install wget

In [ ]:
!pip install ufal.udpipe
!pip install wget

In [ ]:
import wget
import sys

udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
modelfile = wget.download(udpipe_url)

In [ ]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
   entities = {'PROPN'}
   named = False  # переменная для запоминания того, что нам встретилось имя собственное
   memory = []
   mem_case = None
   mem_number = None
   tagged_propn = []

   # обрабатываем текст, получаем результат в формате conllu:
   processed = pipeline.process(text)

   # пропускаем строки со служебной информацией:
   content = [l for l in processed.split('\n') if not l.startswith('#')]

   # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
   tagged = [w.split('\t') for w in content if w]

   for t in tagged:
       if len(t) != 10: # если список короткий — строчка не содержит разбора, пропускаем
           continue
       (word_id,token,lemma,pos,xpos,feats,head,deprel,deps,misc) = t 
       if not lemma or not token: # если слово пустое — пропускаем
           continue
       if pos in entities: # здесь отдельно обрабатываем имена собственные — они требуют особого обращения
           if '|' not in feats:
               tagged_propn.append('%s_%s' % (lemma, pos))
               continue
           morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
           if 'Case' not in morph or 'Number' not in morph:
               tagged_propn.append('%s_%s' % (lemma, pos))
               continue
           if not named:
               named = True
               mem_case = morph['Case']
               mem_number = morph['Number']
           if morph['Case'] == mem_case and morph['Number'] == mem_number:
               memory.append(lemma)
               if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                   named = False
                   past_lemma = '::'.join(memory)
                   memory = []
                   tagged_propn.append(past_lemma + '_PROPN ')
           else:
               named = False
               past_lemma = '::'.join(memory)
               memory = []
               tagged_propn.append(past_lemma + '_PROPN ')
               tagged_propn.append('%s_%s' % (lemma, pos))
       else:
           if not named:
               #if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                #   lemma = num_replace(token)
               tagged_propn.append('%s_%s' % (lemma, pos))
           else:
               named = False
               past_lemma = '::'.join(memory)
               memory = []
               tagged_propn.append(past_lemma + '_PROPN ')
               tagged_propn.append('%s_%s' % (lemma, pos))

   if not keep_punct: # обрабатываем случай, когда пользователь попросил не сохранять пунктуацию (по умолчанию она сохраняется)
       tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
   if not keep_pos:
       tagged_propn = [word.split('_')[0] for word in tagged_propn]
   return tagged_propn

In [ ]:
from ufal.udpipe import Model, Pipeline
import os
import re

udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
udpipe_filename = udpipe_model_url.split('/')[-1]

if not os.path.isfile(modelfile):
       print('UDPipe model not found. Downloading...', file=sys.stderr)
       wget.download(udpipe_model_url)

print('\nLoading the model...', file=sys.stderr)
model = Model.load(modelfile)
process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):

   print('Processing input...', file=sys.stderr)
   lines = text.split('\n')
   tagged = []
   for line in lines:
       # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
       output = process(process_pipeline, text=line)
       tagged_line = ' '.join(output)
       tagged.append(tagged_line)
   return '\n'.join(tagged)

c = 0
for filename in os.listdir("/content/drive/MyDrive/kursovaya/for_modeling"):
  text = open(os.path.join("/content/drive/MyDrive/kursovaya/for_modeling", filename), 'r', encoding='utf-8').read()
  split_regex = re.compile(r'[.|!|?|…]')
  sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(text)])
  sentenced_text = ''
  for s in sentences:
      sentenced_text += s + '\n'
#print(sentenced_text[0:350])
  processed_text = tag_ud(text=sentenced_text, modelfile=modelfile)
#print(processed_text[:350])
  with open('/content/drive/MyDrive/kursovaya/for_training/my_text' + str(c) + '.txt', 'w', encoding='utf-8') as out:
    out.write(processed_text)
    c+=1
    print(c)

In [ ]:
!pip install gensim

In [ ]:
!pip install gensim --upgrade

In [ ]:
import sys
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
f = 'my_text.txt'
data = gensim.models.word2vec.LineSentence(f)

In [ ]:
model = gensim.models.Word2Vec(data, vector_size=500, window=3, min_count=2, sg=0)

In [ ]:
print(len(model.wv.key_to_index))

In [ ]:
import os
from os import cpu_count
from gensim.models import Word2Vec
import gensim
import logging
import time

start = time.time()
data = []
c=0
for c in range(614):
    with open(os.path.join('/content/drive/MyDrive/kursovaya/for_training/my_text' + str(c) + '.txt')):
        data.extend(gensim.models.word2vec.LineSentence('/content/drive/MyDrive/kursovaya/for_training/my_text' + str(c) + '.txt'))
        c+=1
        print(c)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
w2v_model = Word2Vec(data, min_count=1, window=3, vector_size=400, negative=10, alpha=0.03, min_alpha=0.0007, sg=0, workers=cpu_count())
w2v_model.save("word2vec.model")



print(w2v_model.wv.most_similar(positive='солнце_NOUN', topn=5))

print(len(w2v_model.wv.key_to_index))

print(time.time()-start)

In [ ]:
import gensim
from gensim.models import Word2Vec

model_21 = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/220/model.bin', binary = True)
model_19 = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/kursovaya/model.bin', binary = True)
my_model = Word2Vec.load('word2vec.model')

In [ ]:
word = 'каменный_ADJ'
print(model_21.most_similar(positive=word, topn=5))
print(model_19.most_similar(positive=word, topn=5))
print(my_model.wv.most_similar(positive=word, topn=5))

[('деревянный_ADJ', 0.789763867855072), ('кирпичный_ADJ', 0.6676967144012451), ('белокаменный_ADJ', 0.6330768465995789), ('гранитный_ADJ', 0.6321970224380493), ('бревенчатый_ADJ', 0.6212176084518433)]
[('гранитный_ADJ', 0.6750349402427673), ('деревянный_ADJ', 0.6713505387306213), ('тесаный_ADJ', 0.6575525403022766), ('кирпичный_ADJ', 0.6524932980537415), ('чугунный_ADJ', 0.6057920455932617)]
[('глиняный_ADJ', 0.6837567687034607), ('гранитный_ADJ', 0.675308346748352), ('земляной_ADJ', 0.6619945764541626), ('деревянный_ADJ', 0.6281141042709351), ('массивный_ADJ', 0.6235159635543823)]
